###  Created by Luis A. Sanchez-Perez (alejand@umich.edu).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

Builds an MLP classifier using borh sklearn and keras (tensorflow backend) implementations

### Libraries

In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
import tensorflow as tf
import os

### General use functions

In [2]:
def compute_metrics(mdl,X,y):
    threshold = 0.5
    y_pred = mdl.predict(X)
    y_pred[y_pred >= threshold] = 1
    y_pred[y_pred < threshold] = 0
    print('Accuracy: ', '%.2f'  % accuracy_score(y, y_pred))
    print('F1 Score: ', '%.2f'  % f1_score(y, y_pred, average='macro'))
    print('Recall: ', '%.2f'  % recall_score(y, y_pred, average='macro'))
    print('Precision: ', '%.2f'  % precision_score(y, y_pred, average='macro'))    

### Loads a prepares dataset

In [3]:
# Loads dataset from file
dataset = pd.read_csv('../../datasets/classification/bank_exiting.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values
# Encoding categorical data
encoder_x1 = LabelEncoder()
X[:,1] = encoder_x1.fit_transform(X[:,1]) # only 0 or 1 after this (just one column needed)
encoder_x2 = LabelEncoder()
X[:,2] = encoder_x2.fit_transform(X[:,2]) # more than two possible values (need to encode this using onehotencoder)
ct = ColumnTransformer([('country_category', OneHotEncoder(categories='auto'),[1])], remainder='passthrough')
X = ct.fit_transform(X)
X = X[:,1:]
# Splits dataset into training/test sets
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Trains a MLP classifier using sklearn implementation

In [4]:
print('Training size: ', X_train.shape)
print('Test size: ', X_test.shape)
start = time.perf_counter()
regularization = 1
mdl = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(20,6,), activation='logistic', max_iter=1000,alpha=regularization)
mdl.fit(X_train,y_train)
end = time.perf_counter()
print('\nTime training: ', end - start, ' sec')
print('# of iterations: ', mdl.n_iter_ - 1)
print('\nTraining Metrics: ')
compute_metrics(mdl,X_train,y_train)
print('\nTest Metrics: ')
compute_metrics(mdl,X_test,y_test)

Training size:  (7000, 11)
Test size:  (3000, 11)

Time training:  3.7822581999999993  sec
# of iterations:  703

Training Metrics: 
Accuracy:  0.88
F1 Score:  0.79
Recall:  0.76
Precision:  0.84

Test Metrics: 
Accuracy:  0.86
F1 Score:  0.76
Recall:  0.74
Precision:  0.79


### Trains a MLP classifier using Tensorflow and Keras

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
start = time.perf_counter()
regularization = 0
mdl = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='sigmoid', input_dim=X_train.shape[1],
                          kernel_regularizer=tf.keras.regularizers.l2(regularization)),
    tf.keras.layers.Dense(6, activation='sigmoid',
                          kernel_regularizer=tf.keras.regularizers.l2(regularization)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
mdl.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = mdl.fit(X_train, y_train, epochs=100, verbose=2)
end = time.perf_counter()

Train on 7000 samples
Epoch 1/100
7000/7000 - 0s - loss: 0.5162 - accuracy: 0.7977
Epoch 2/100
7000/7000 - 0s - loss: 0.4828 - accuracy: 0.7977
Epoch 3/100
7000/7000 - 0s - loss: 0.4636 - accuracy: 0.7977
Epoch 4/100
7000/7000 - 0s - loss: 0.4451 - accuracy: 0.7984
Epoch 5/100
7000/7000 - 0s - loss: 0.4337 - accuracy: 0.8103
Epoch 6/100
7000/7000 - 0s - loss: 0.4287 - accuracy: 0.8113
Epoch 7/100
7000/7000 - 0s - loss: 0.4262 - accuracy: 0.8114
Epoch 8/100
7000/7000 - 0s - loss: 0.4245 - accuracy: 0.8134
Epoch 9/100
7000/7000 - 0s - loss: 0.4230 - accuracy: 0.8160
Epoch 10/100
7000/7000 - 0s - loss: 0.4214 - accuracy: 0.8183
Epoch 11/100
7000/7000 - 0s - loss: 0.4197 - accuracy: 0.8211
Epoch 12/100
7000/7000 - 0s - loss: 0.4177 - accuracy: 0.8231
Epoch 13/100
7000/7000 - 0s - loss: 0.4156 - accuracy: 0.8253
Epoch 14/100
7000/7000 - 0s - loss: 0.4134 - accuracy: 0.8269
Epoch 15/100
7000/7000 - 0s - loss: 0.4113 - accuracy: 0.8289
Epoch 16/100
7000/7000 - 0s - loss: 0.4089 - accuracy: 0.

In [6]:
print('\nTime training: ', end - start, ' sec')
print('# of iterations: ', history.epoch[-1] + 1)
print('\nTraining Metrics: ')
compute_metrics(mdl,X_train,y_train)
print('\nTest Metrics: ')
compute_metrics(mdl,X_test,y_test)


Time training:  17.0939734  sec
# of iterations:  100

Training Metrics: 
Accuracy:  0.87
F1 Score:  0.75
Recall:  0.72
Precision:  0.82

Test Metrics: 
Accuracy:  0.86
F1 Score:  0.77
Recall:  0.74
Precision:  0.81
